In [2]:
from langchain.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA

In [3]:
pdf_loader = DirectoryLoader('../../training_data/', glob="**/*.pdf")
#txt_loader = DirectoryLoader('../../training_data/', glob="**/*.txt")
#word_loader = DirectoryLoader('../../training_data/', glob="**/*.docx")
csv_loader = DirectoryLoader('../../training_data/', glob="**/*.csv")

In [4]:
from bs4 import BeautifulSoup as Soup

url = "https://developers.hubspot.com/docs/api/overview"
recursive_url_loader = RecursiveUrlLoader(url=url, max_depth=500, extractor=lambda x: Soup(x, "html.parser").text)

loaders = [recursive_url_loader, csv_loader, pdf_loader]
documents = []
for loader in loaders:
    documents.extend(loader.load())

print(f"Total number of documents: {len(documents)}")

Total number of documents: 5


In [5]:
import os
from dotenv import load_dotenv

load_dotenv()
huggingfacehub_api_key= os.getenv('HUGGINGFACEHUB_API_TOKEN')
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain.embeddings import CohereEmbeddings

In [6]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
texts = text_splitter.split_documents(documents)
embeddings = CohereEmbeddings()

Created a chunk of size 1825, which is longer than the specified 1000
Created a chunk of size 1825, which is longer than the specified 1000


In [7]:
db = Chroma.from_documents(texts, embeddings, collection_name="hubspot_hubl_knowledge_base")

In [8]:
from langchain.chat_models import ChatLiteLLM
from langchain.prompts.chat import (
    ChatPromptTemplate
)

In [9]:
import os
from dotenv import load_dotenv

load_dotenv()
from langchain.llms import Cohere
cohere_api_key= os.getenv('COHERE_API_KEY')
cohere_llm = ChatLiteLLM(model="command-nightly")


In [10]:
import os
from dotenv import load_dotenv
from langchain.llms import GooglePalm

load_dotenv()
google_api_key= os.getenv('GOOGLE_API_KEY')
google_palm_llm = GooglePalm()
google_palm_llm.temperature = 0.1


In [11]:
hubl_docs_chain = RetrievalQA.from_chain_type(
    llm=cohere_llm, chain_type="stuff", retriever=db.as_retriever()
)

In [12]:
# Import things that are needed generically
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.utilities import PythonREPL
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.tools.file_management import (
    ReadFileTool,
    CopyFileTool,
    DeleteFileTool,
    MoveFileTool,
    WriteFileTool,
    ListDirectoryTool,
)
from langchain.tools import ShellTool
from langchain.agents import load_tools

In [13]:
#DuckDuckGo Class
duckduck_search = DuckDuckGoSearchRun()
shell_tool = ShellTool()
#Python REPL Class
python_repl = PythonREPL()

In [14]:
tools = [
    Tool(
        name="HubSpot Docs",
        func=hubl_docs_chain.run,
        description="Useful for when you need to answer questions about HubSpot. Output should be code or information about HubSpot's API",
    ),
    Tool(
        name = "DuckDuckGo",
        func=duckduck_search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    Tool(
    name="shell",
    description="This tool gives you access to a system's shell. Can be used to run shell commands",
    func=shell_tool.run
    )
]

In [15]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, google_palm_llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True
)

In [16]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor

In [17]:
prefix = """You're a seasoned software developer with advanced knowledge of HubSpot's Documentation and APIs."""
suffix = """Take a deep breath and solve this problem step-by-step."

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, prefix=prefix, suffix=suffix, input_variables=["input", "agent_scratchpad"]
)

In [18]:
from langchain.chains import LLMChain

In [19]:
llm_chain = LLMChain(llm=google_palm_llm, prompt=prompt)

In [20]:
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

In [21]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [22]:
agent_executor.run("Create a script to create my own custom workflow")



> Entering new AgentExecutor chain...
Thought: I need to find the documentation for creating custom workflows
Action: HubSpot Docs
Action Input: /docs/workflows/create-custom-workflow
Observation:  I'm sorry, but I cannot find any relevant information regarding the requested URL /docs/workflows/create-custom-workflow. 

Note that this information may not be up to date, as I only have knowledge up to January 2023. If you'd like more accurate and real-time results, I recommend using a search engine or visiting the HubSpot website directly. 

Is there anything else I can help you find on the internet? 
Thought:I need to find the documentation for creating custom workflows
Action: DuckDuckGo
Action Input: create custom workflow hubspot
Observation: To get started creating a workflow: In your HubSpot account, navigate to Automation > Workflows . You can create a new workflow from scratch or from a template: In your HubSpot account, navigate to Automation > Workflows. Click the name of an 

'Agent stopped due to iteration limit or time limit.'